# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read the weather data into a dataframe
city_weather = pd.read_csv("Output_Data/city_weather.csv")

#display dataframe
city_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mossamedes,-15.1961,12.1522,61.97,87,35,6.64,AO,1635125644
1,Busselton,-33.6500,115.3333,62.04,60,97,8.81,AU,1635125644
2,Flinders,-34.5833,150.8552,72.18,47,5,5.01,AU,1635125645
3,Grindavik,63.8424,-22.4338,44.55,100,90,3.44,IS,1635125645
4,Port Alfred,-33.5906,26.8910,56.77,79,0,11.03,ZA,1635125636


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
#heatmap of the humidity
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]
fig = gmaps.figure()

#add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = city_weather.loc[
    (city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80) & \
    (city_weather["Wind Speed"] < 10) & (city_weather ["Cloudiness"]==0)
    
].dropna()

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,Salalah,17.0151,54.0924,73.49,64,0,4.61,OM,1635125667
111,San Miguel de Tucumán,-26.8241,-65.2226,73.40,33,0,6.91,AR,1635125414
202,Arlit,18.7369,7.3853,78.93,13,0,1.03,NE,1635125715
242,Chotila,22.4167,71.1833,73.38,68,0,5.93,IN,1635125730
309,Saint-Pierre,-21.3393,55.4781,73.04,77,0,4.61,RE,1635125673
343,Ishurdi,24.1333,89.0833,77.07,71,0,2.48,BD,1635125764
358,Maceió,-9.6658,-35.7353,78.24,94,0,5.75,BR,1635125770
482,Sultānpur,26.2667,82.0667,72.79,69,0,3.80,IN,1635125818
574,Karachi,24.9056,67.0822,71.42,56,0,4.61,PK,1635125720


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#create hotel dataframe
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng"]]

#add "hotel name" column
hotel_df["Hotel Name"] = ""

#display dataframe
hotel_df

<ipython-input-7-f1ee86e65a1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
75,Salalah,OM,17.0151,54.0924,
111,San Miguel de Tucumán,AR,-26.8241,-65.2226,
202,Arlit,NE,18.7369,7.3853,
242,Chotila,IN,22.4167,71.1833,
309,Saint-Pierre,RE,-21.3393,55.4781,
343,Ishurdi,BD,24.1333,89.0833,
358,Maceió,BR,-9.6658,-35.7353,
482,Sultānpur,IN,26.2667,82.0667,
574,Karachi,PK,24.9056,67.0822,


In [16]:
#set the parameters to search for hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#loop through the dataframe using iterrows()
for index, row in hotel_df.iterrows():
    #get the lat and lng from the current row
    lat = row["Lat"]
    lng = row["Lng"]

    #use the lat and lng to set the location value in the params
    params["location"] = f"{lat},{lng}"
    
    #use the googlee apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and print the url
    name_address = requests.get(base_url, params=params).json()
    
    try:
        #populate the hotel name column
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']} - skipping")

hotel_df

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
75,Salalah,OM,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
111,San Miguel de Tucumán,AR,-26.8241,-65.2226,Dallas Hotel Tucuman
202,Arlit,NE,18.7369,7.3853,Case de Passage De la Prefecture
242,Chotila,IN,22.4167,71.1833,Hotel Honest
309,Saint-Pierre,RE,-21.3393,55.4781,Lindsey Hôtel
343,Ishurdi,BD,24.1333,89.0833,Rafiur Rahman
358,Maceió,BR,-9.6658,-35.7353,Poutur Pousada
482,Sultānpur,IN,26.2667,82.0667,OYO 49568 Vrindavan Guest House
574,Karachi,PK,24.9056,67.0822,Sadaat


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))